# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install requests schedule BeautifulSoup4 pytest 

In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import re
import json
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from typing import Optional
from urllib.parse import urljoin


## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
 def get_book_data(book_url: str) -> dict:
    """
    Забирает со страницы книги (Books to Scrape) основные поля и таблицу Product Information.

    Параметры
    ---------
    book_url : str
        Полный или относительный URL страницы конкретной книги.

    Возвращает
    ----------
    dict
        {
          'title': str | None,
          'price': float | None,          # цена как число
          'price_text': str | None,       # исходная строка цены (с символом валюты)
          'rating': int | None,           # 1..5
          'availability': int | None,     # сколько штук «In stock»
          'description': str | None,
          'product_info': dict[str, str]  # пара th -> td из таблицы
        }
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ


    try:
        import requests
        from bs4 import BeautifulSoup
    except Exception as e:
        # Если в окружении нет зависимостей — вернём «пустой» результат с подсказкой
        return {
            'title': None, 'price': None, 'price_text': None, 'rating': None,
            'availability': None, 'description': None, 'product_info': {},
            '_error': f'Импорт зависимостей не удался: {e}. Установите bs4 и requests.'
        }

    def _to_float_money(s: Optional[str]) -> Optional[float]:
        if not s:
            return None
        # Оставим точку как разделитель, выкинем всё кроме цифр/точки/минуса
        s = s.replace(',', '.')
        m = re.search(r'-?\d+(?:\.\d+)?', s)
        return float(m.group()) if m else None

    def _rating_to_int(tag) -> Optional[int]:
        # На сайте рейтинг кодируется классом: "star-rating Three"
        mapping = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
        if not tag:
            return None
        for cls in tag.get('class', []):
            if cls in mapping:
                return mapping[cls]
        return None

    def _availability_to_int(s: Optional[str]) -> Optional[int]:
        if not s:
            return None
        m = re.search(r'(\d+)', s)
        return int(m.group(1)) if m else 0

    # 1) Скачать страницу
    resp = requests.get(book_url, timeout=15, headers={'User-Agent': 'Mozilla/5.0'})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    # 2) Основные поля
    product_main = soup.select_one('.product_main')

    title = product_main.select_one('h1').get_text(strip=True) if product_main else None

    price_text = None
    price = None
    price_tag = product_main.select_one('.price_color') if product_main else None
    if price_tag:
        price_text = price_tag.get_text(strip=True)
        price = _to_float_money(price_text)

    rating = _rating_to_int(product_main.select_one('.star-rating')) if product_main else None

    availability_text = None
    availability = None
    avail_tag = product_main.select_one('.instock.availability') if product_main else None
    if avail_tag:
        availability_text = ' '.join(avail_tag.get_text(separator=' ', strip=True).split())
        availability = _availability_to_int(availability_text)

    # 3) Описание
    description = None
    desc_header = soup.select_one('#product_description')
    if desc_header:
        p = desc_header.find_next('p')
        if p:
            description = p.get_text(strip=True) or None

    # 4) Таблица Product Information -> dict
    product_info = {}
    for row in soup.select('table.table.table-striped tr'):
        th = row.find('th')
        td = row.find('td')
        if th and td:
            key = th.get_text(strip=True)
            val = td.get_text(" ", strip=True)
            product_info[key] = val

    # (опционально) Нормализуем ключ «Availability» из таблицы, если он есть
    if 'Availability' in product_info and availability is None:
        availability = _availability_to_int(product_info.get('Availability'))

    return {
        'title': title,
        'price': price,
        'price_text': price_text,
        'rating': rating,
        'availability': availability,
        'description': description,
        'product_info': product_info
    }

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': 51.77,
 'price_text': 'Â£51.77',
 'rating': 3,
 'availability': 22,
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place 

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(catalog_page1_url: str,
                 is_save: bool = False,
                 return_json: bool = False,
                 page_count: int = 0,
                 verbose: bool = True):
    """
    Обходит страницы каталога Books to Scrape, собирает ссылки на книги,
    парсит каждую книгу через get_book_data и печатает подробную статистику.
    """
    import requests
    from bs4 import BeautifulSoup

    def _mk_page_url(n: int) -> str:
        return re.sub(r'page-\d+\.html', f'page-{n}.html', catalog_page1_url)

    headers = {"User-Agent": "Mozilla/5.0"}
    timeout = 15

    # ====== ШАГ 1. СБОР ССЫЛОК ======
    t0 = time.time()
    page = 1
    book_urls = []

    while True:
        url = _mk_page_url(page)
        r = requests.get(url, headers=headers, timeout=timeout)
        if r.status_code == 404:
            break
        r.raise_for_status()

        soup = BeautifulSoup(r.text, "html.parser")
        links = [urljoin(url, a.get("href")) for a in soup.select("article.product_pod h3 a")]
        book_urls.extend(links)

        if verbose:
            print(f"Обработка страницы №{page}")

        page += 1
        if page_count and page > page_count:
            if verbose:
                print(f"Достигнута страница: {page_count}")
            break

    t1 = time.time()
    if verbose:
        print(f"Время обработки ссылок: {t1 - t0:.2f} сек.")
        print(f"Всего найдено URL книг для парсинга: {len(book_urls)}")
        print("Парсю...")

    # ====== ШАГ 2. ПАРСИНГ КНИГ ======
    results = []
    t2 = time.time()
    for i, url in enumerate(book_urls, 1):
        try:
            data = get_book_data(url)
            if isinstance(data, dict):
                data.setdefault("_source_url", url)
            results.append(data)
        except Exception as e:
            results.append({"_source_url": url, "_error": f"{type(e).__name__}: {e}"})
        # при желании можно печатать прогресс каждые 100 книг
        # if verbose and i % 100 == 0: print(f"Спарсено книг: {i}/{len(book_urls)}")

    t3 = time.time()

    if verbose:
        print(f"Время парсинга книг: {t3 - t2:.2f} сек.")
        print(f"Обработано книг: {len(results)}")
        print(f"Общее время: {t3 - t0:.2f} сек.")
        if t3 - t2 > 0:
            print(f"Средняя скорость: {len(results) / (t3 - t2):.2f} книг/сек")

    # ====== ШАГ 3. СОХРАНЕНИЕ ======
    if is_save:
        out_dir = Path.cwd() / "artifacts"     # сохраняем в ./artifacts
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / "books_data.txt"

        with out_path.open("w", encoding="utf-8") as f:
            if return_json:
                f.write(json.dumps(results, ensure_ascii=False))
            else:
                # JSON Lines (по строке на книгу)
                for row in results:
                    f.write(json.dumps(row, ensure_ascii=False) + "\n")

    return json.dumps(results, ensure_ascii=False) if return_json else results


In [6]:
catalog_url = 'http://books.toscrape.com/catalogue/page-1.html'
res = scrape_books(catalog_url, is_save=True, return_json=True, page_count = 0) # Допишите ваши аргументы
print(type(res), len(res)) # проверки 

Обработка страницы №1
Обработка страницы №2
Обработка страницы №3
Обработка страницы №4
Обработка страницы №5
Обработка страницы №6
Обработка страницы №7
Обработка страницы №8
Обработка страницы №9
Обработка страницы №10
Обработка страницы №11
Обработка страницы №12
Обработка страницы №13
Обработка страницы №14
Обработка страницы №15
Обработка страницы №16
Обработка страницы №17
Обработка страницы №18
Обработка страницы №19
Обработка страницы №20
Обработка страницы №21
Обработка страницы №22
Обработка страницы №23
Обработка страницы №24
Обработка страницы №25
Обработка страницы №26
Обработка страницы №27
Обработка страницы №28
Обработка страницы №29
Обработка страницы №30
Обработка страницы №31
Обработка страницы №32
Обработка страницы №33
Обработка страницы №34
Обработка страницы №35
Обработка страницы №36
Обработка страницы №37
Обработка страницы №38
Обработка страницы №39
Обработка страницы №40
Обработка страницы №41
Обработка страницы №42
Обработка страницы №43
Обработка страницы №

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [10]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
import schedule
import time

catalog_url = "http://books.toscrape.com/catalogue/page-1.html"

# Настраиваем ежедневный запуск функции
schedule.clear()
schedule.every().day.at("19:00").do(
    scrape_books,
    catalog_url,
    is_save=True,
    return_json=True,
    page_count=5,
)

# Бесконечный цикл ожидания по расписанию
while True:
    schedule.run_pending()
    time.sleep(5)

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Обработка страницы №1
Обработка страницы №2
Обработка страницы №3
Обработка страницы №4
Обработка страницы №5
Достигнута страница: 5
Время обработки ссылок: 1.97 сек.
Всего найдено URL книг для парсинга: 100
Парсю...
Время парсинга книг: 25.63 сек.
Обработано книг: 100
Общее время: 27.60 сек.
Средняя скорость: 3.90 книг/сек


KeyboardInterrupt: 

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [11]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! cd ../../Desktop/hw-books-parser && set PYTHONPATH=. && pytest -v tests/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.12.7, pytest-7.4.4, pluggy-1.0.0 -- C:\Users\Theodor\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Theodor\Desktop\hw-books-parser
plugins: anyio-4.2.0
collecting ... collected 4 items

tests/test_scraper.py::test_get_book_data_structure_and_title PASSED     [ 25%]
tests/test_scraper.py::test_scrape_books_count_by_pages[1-20] PASSED     [ 50%]
tests/test_scraper.py::test_scrape_books_count_by_pages[2-40] PASSED     [ 75%]
tests/test_scraper.py::test_scrape_books_saves_file PASSED               [100%]

============================= 4 passed in 23.29s ==============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```